In [61]:
from astropy_healpix import HEALPix
from astropy.coordinates import Galactic
from astropy.io import fits
import numpy as np
from scipy.interpolate import griddata
import matplotlib.pyplot as plt

In [48]:
hp = HEALPix(nside=256, frame=Galactic())
npix = hp.npix
lon, lat = hp.healpix_to_lonlat(np.arange(npix))
lon = lon.degree  #convert to degrees
lat = lat.degree

In [49]:
hdu1 = fits.open('VIII_28/fits/whsky001.fit')
header = hdu1[0].header
NAXIS1 = header['NAXIS1']
NAXIS2 = header['NAXIS2']
CRVAL1 = header['CRVAL1']
CRVAL2 = header['CRVAL2']
CRPIX1 = header['CRPIX1']
CRPIX2 = header['CRPIX2']
CDELT1 = header['CDELT1']
CDELT2 = header['CDELT2']
hdu1.close()

In [50]:
bell_lon = CRVAL1 + CDELT1 * (np.arange(NAXIS1) - CRPIX1 + 1)
bell_lat = CRVAL2 + CDELT2 * (np.arange(NAXIS2) - CRPIX2 + 1)
bell_lon, bell_lat = np.meshgrid(bell_lon, bell_lat)
bell_lon = bell_lon.flatten()
bell_lat = bell_lat.flatten()

In [51]:
nchan = 145
bell_hp = np.zeros((npix, nchan))

In [52]:
pattern = "VIII_28/fits/whsky{:03}.fit"
for ichan in range(nchan):
    filename = pattern.format(ichan + 1)
    hdu = fits.open(filename)
    image = hdu[0].data.flatten()
    hdu.close()

    interpolated_values = griddata((bell_lon, bell_lat), image, (lon, lat), method='linear')
    bell_hp[:, ichan] = interpolated_values

In [54]:
hdu = fits.PrimaryHDU(bell_hp)
hdul = fits.HDUList([hdu])
hdul.writeto('output_healpix_map.fits', overwrite=True)

In [105]:
def compare_fits_edges(file_path):
    with fits.open(file_path) as hdul:
        data = hdul[0].data

    top_row = data[0, :]
    bottom_row = data[-1, :]
    left_column = data[:, 0]
    right_column = data[:, -1]

    rows_equal = np.array_equal(top_row, bottom_row)
    columns_equal = np.array_equal(left_column, right_column)

    rows_equal_with_nan = np.all((top_row == bottom_row) | (np.isnan(top_row) & np.isnan(bottom_row)))
    columns_equal_with_nan = np.all((left_column == right_column) | (np.isnan(left_column) & np.isnan(right_column)))

    return rows_equal, columns_equal, rows_equal_with_nan, columns_equal_with_nan

file_path = 'VIII_28/fits/whsky001.fit'
rows_equal, columns_equal, rows_equal_with_nan, columns_equal_with_nan = compare_fits_edges(file_path)

print(f"Rows equal (without considering NaNs): {rows_equal}")
print(f"Columns equal (without considering NaNs): {columns_equal}")
print(f"Rows equal (considering NaNs): {rows_equal_with_nan}")
print(f"Columns equal (considering NaNs): {columns_equal_with_nan}")



Rows equal (without considering NaNs): False
Columns equal (without considering NaNs): False
Rows equal (considering NaNs): True
Columns equal (considering NaNs): False
